<a href="https://colab.research.google.com/github/acoiman/pdt/blob/main/asthma_mortality/notebooks/Python/07.Asthma_Mortality_LULC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Land Use and Land Cover (LULC) Changes Data

In this notebook, we will calculate Land Use and Land Cover (LULC) transition areas for consecutive years from 2001 to 2022. Specifically, we will focus on transitions involving:

i) Agricultural and livestock areas (AGR)

ii) Natural wooded vegetation (NWV)

iii) Built-up (BU)

To get  AGR and NWV transition areas, we will use the dataset provided by [MapBiomas Argentina](https://argentina.mapbiomas.org/mapas-de-la-coleccion/). We will also use [GLAD dataset ](https://glad.umd.edu/dataset) to extract BU transition areas.


##📦 Import Required Libraries

In [ ]:
# geospatial data handling
import ee
import geemap
import geopandas as gpd
import folium

# data frame libraries
import pandas as pd

# paralell execution
from joblib import Parallel, delayed

# other libraries
import branca.colormap as cm
import os
from itables import init_notebook_mode

## 🌍 Connect to Google Earth Engine (GEE)

In [ ]:
# trigger the authentication flow
ee.Authenticate()

In [ ]:
# initialize the library.
ee.Initialize(project='ee-pdt')
print(ee.String('Hello from the Earth Engine servers!').getInfo())

In [ ]:
# Set the PROJ_LIB path
os.environ['PROJ_LIB'] = "/opt/conda/envs/gds/share/proj"

In [ ]:
# change to my computer home directory
%cd work/

## 📟 Calculating the Normalized Agricultural and Livestock Transition areas (NAGRT)

### Computing NAGRT for the period 2000-2001

In this section, we will calculate the agricultural and livestock transition areas for the period 2000–2001. This includes all areas that changed from any land cover class to the agricultural and livestock category between 2000 and 2001 (gainings). The resulting dataset will be aggregated by department, normalized by each department’s surface area, and then multiplied by 1,000 km² to enhance interpretability.

In [ ]:
# Load the transition image and select the 2000–2001 band
transitions = ee.Image(
    'projects/mapbiomas-public/assets/argentina/collection1/mapbiomas_argentina_collection1_transitions_v1'
).select('transitions_2000_2001')

# Define the target land cover classes (to transition into)
target_classes = [18, 15, 9, 36, 21]  # Agriculture, Pasture, Forest Plant., Shrub Plant., Mosaic

# Extract FROM and TO class
from_class = transitions.divide(100).floor().toInt()
to_class = transitions.mod(100).toInt()

# Build condition: TO in target_classes
to_is_target = to_class.eq(target_classes[0])
for cls in target_classes[1:]:
    to_is_target = to_is_target.Or(to_class.eq(cls))

# Build condition: FROM NOT in target_classes
from_not_target = from_class.neq(target_classes[0])
for cls in target_classes[1:]:
    from_not_target = from_not_target.And(from_class.neq(cls))

# Final mask: to target class, but from another class
expansion_mask = to_is_target.And(from_not_target)

# Binary mask
binary_mask = expansion_mask.selfMask()

# Load departments
departments = ee.FeatureCollection('projects/ee-pdt/assets/argentina/ar_dpto')

# Pixel area in km²
pixel_area_km2 = ee.Image.pixelArea().divide(1e6)

# Area of valid pixels
expansion_area_img = binary_mask.multiply(pixel_area_km2)

# Function to compute and normalize expansion area
def compute_expansion(feature):
    area_exp = expansion_area_img.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=feature.geometry(),
        scale=30,
        maxPixels=1e13
    ).get('transitions_2000_2001')

    dept_area_km2 = feature.geometry().area().divide(1e6)
    exp_km2 = ee.Number(area_exp)
    exp_norm = exp_km2.divide(dept_area_km2)

    exp_norm_1000 = exp_norm.multiply(1000).multiply(100).round().divide(100)

    return feature.set({
        'NAGRT_0001': exp_norm_1000,
    })

In [ ]:
# Apply function to all departments
result = departments.map(compute_expansion)

In [ ]:
# Convert FeatureCollection into a GeoDataFrame
gdf_nagrt_0001 = geemap.ee_to_gdf(result)

In [ ]:
# visualize the dataframe
init_notebook_mode(all_interactive=True)
gdf_nagrt_0001.head()

#### Mapping the NAGRT period 2000-2001

In this section we will display vector data (GeoDataFrame with NAGRT 2000-2001) using folium.Choropleth

In [ ]:
# Create base map centered on Argentina
m = folium.Map(location=[-38.4, -63.6], zoom_start=5, control_scale=True)

# Add Choropleth layer from gdf
choropleth = folium.Choropleth(
    geo_data=gdf_nagrt_0001,
    name='NAGRT 2000-2001',
    data=gdf_nagrt_0001.drop(columns=['geometry']),
    columns=['IDDPTO', 'NAGRT_0001'],
    key_on='feature.properties.IDDPTO',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.3,
    nan_fill_color='gray',
    legend_name='Normalized Agricultural and Livestock Area transitions (NAGRT) period 2000-2021'
)
choropleth.add_to(m)

# Now add an interactive GeoJson layer with popups
geojson = folium.GeoJson(
    gdf_nagrt_0001,
    name='Interactive Layer',
    style_function=lambda feature: {
        'fillOpacity': 0,
        'color': 'black',
        'weight': 0.3,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['IDDPTO', 'NAGRT_0001'],
        aliases=['Dept ID:', 'Expansion (/1000 km²):'],
        localize=True,
        sticky=True,
        labels=True
    )
).add_to(m)

# Display expansion raster (binary_mask) as red overlay ---
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['#ffffff', '#000000']  # white = no change, red = expansion
}

# Get map tiles from Earth Engine image
map_id_dict = binary_mask.getMapId(vis_params)

# Create Folium TileLayer
tile = folium.raster_layers.TileLayer(
    tiles=map_id_dict['tile_fetcher'].url_format,
    attr='Earth Engine',
    name='Mapbiomas transition Raster (Black)',
    overlay=True,
    control=True
)

# Add raster layer to folium map
m.add_child(tile)

# Add layer control
folium.LayerControl().add_to(m)

In [ ]:
# display the map
m

####  Checking results of NAGRT period 2000-2001

In this section, we will extract the transition image where binary_mask is equal to 1 for the department with the highest NAGRT (IDDPTO = "06651"). We will then verify whether pixel values correspond to transitions that end in, but do not start from any agricultural and livestock area class code (15, 18, 9, 36, 21)

In [ ]:
# Select the feature with IDDPTO '06651'
dept_06651 = departments.filter(ee.Filter.eq('IDDPTO', '06651')).first()

# Clip the transitions image by the geometry of the selected department
transitions_clipped = transitions.clip(dept_06651.geometry())
binary_mask_clipped = binary_mask.clip(dept_06651.geometry())

# Get pixel values of the transitions image where binary_mask_clipped == 1
# Define the reducer to get a list of pixel values
reducer = ee.Reducer.toList()

# Reduce the clipped transitions image using the clipped binary mask
# We multiply the transitions image by the binary mask so only pixels where
# the mask is 1 are included in the reduction.
pixel_values = transitions_clipped.updateMask(binary_mask_clipped).reduceRegion(
    reducer=reducer,
    geometry=dept_06651.geometry(),
    scale=30,  # Use the same scale as before
    maxPixels=1e13
)

# Extract the list of pixel values
list_of_values = pixel_values.get('transitions_2000_2001').getInfo()

In [ ]:
# get unique values of list_of_values
# the result meets our checking condition
unique_values = list(set(list_of_values))
unique_values

###  Computing NAGRT from 2000 to 2022

In this section, we will calculate the agricultural and livestock transition areas for consecutive years. This includes all areas that changed from any land cover class to the agricultural and livestock category during each year-to-year transition between 2000 and 2022 (gainings). The resulting dataset will be aggregated by department, normalized by each department’s surface area, and multiplied by 1,000 km² to enhance interpretability.

In [ ]:
# Load the FeatureCollection of departments in Argentina
departments = ee.FeatureCollection('projects/ee-pdt/assets/argentina/ar_dpto')

# Create an image representing the area of each pixel in square kilometers
pixel_area_km2 = ee.Image.pixelArea().divide(1e6)

# Define the target classes for analysis or processing
target_classes = [18, 15, 9, 36, 21]

In [ ]:
def process_transition_year(y1):
    """
    Compute the Normalized Agricultural and Livestock Transition Rate (NAGRT)
    for a specific year-to-year transition and return it as a GeoDataFrame.

    Parameters:
        y1 (int): The start year of the transition (e.g., 2001 for 2001–2002)

    Returns:
        GeoDataFrame with columns: ['IDDPTO', 'NAGRT_y1_y2']

    Note: The name of the output column should be NAGRT_y1_y2,
          but for the sake of posterior analysis we choose NAGRT_y2
    """
    try:

        # Define transition band and output column name
        y2 = y1 + 1
        band_name = f"transitions_{y1}_{y2}"
        # col_name = f"NAGRT_{y1}_{y2}"
        col_name = f"NAGRT_{y2}"

        # Load the transition image band for the given year
        transitions = ee.Image('projects/mapbiomas-public/assets/argentina/collection1/mapbiomas_argentina_collection1_transitions_v1') \
            .select(band_name)

        # Extract FROM and TO classes from the transition code
        from_class = transitions.divide(100).floor().toInt()
        to_class = transitions.mod(100).toInt()

        # Build mask for transitions TO target classes
        to_is_target = to_class.eq(target_classes[0])
        for cls in target_classes[1:]:
            to_is_target = to_is_target.Or(to_class.eq(cls))

        # Exclude transitions FROM the same target classes (i.e., exclude stable)
        from_not_target = from_class.neq(target_classes[0])
        for cls in target_classes[1:]:
            from_not_target = from_not_target.And(from_class.neq(cls))

        # Final mask: to a target class, but not from a target class
        expansion_mask = to_is_target.And(from_not_target)

        # Binary mask image where valid transitions are 1
        binary_mask = expansion_mask.selfMask()

        # Multiply by pixel area in km² to get expansion area per pixel
        expansion_area_img = binary_mask.multiply(pixel_area_km2)

        # Per-department computation function
        def compute_metrics(feature):
            area_exp = expansion_area_img.reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=feature.geometry(),
                scale=30,
                maxPixels=1e13
            ).get(band_name)

            dept_area_km2 = feature.geometry().area().divide(1e6)
            exp_km2 = ee.Number(area_exp)
            exp_norm = exp_km2.divide(dept_area_km2)
            exp_norm_1000 = exp_norm.multiply(1000).multiply(100).round().divide(100)

            return feature.set({col_name: exp_norm_1000})

        # Apply computation to all departments and select relevant fields
        result_fc = departments.map(compute_metrics).select(['IDDPTO', col_name])

        # Convert to GeoDataFrame
        gdf = geemap.ee_to_gdf(result_fc)

        # Return DataFrame with only ID and calculated column
        return gdf[['IDDPTO', col_name]]

    except Exception as e:
        print(f"[ERROR] Processing {y1}-{y2}: {e}")
        return pd.DataFrame(columns=['IDDPTO', f"NAGRT_{y1}_{y2}"])

In [ ]:
# list to store all dataframe derived from the for loop
all_dfs_agr = []

In [ ]:
# Loop from 2000 to 2021 (ending at transitions_2021_2022)
for year in range(2000, 2022):
    print(f"Processing transition {year}-{year+1}")
    df = process_transition_year(year)
    all_dfs_agr.append(df)

In [ ]:
# Merge all dataframes on 'IDDPTO'
df_final_agr = all_dfs_agr[0]
for df in all_dfs_agr[1:]:
    df_final_agr = df_final_agr.merge(df, on='IDDPTO', how='outer')

In [ ]:
init_notebook_mode(all_interactive=True)
df_final_agr.head()

In [ ]:
df_final_agr.info()

In [ ]:
# test if df_final_agr['NAGRT_2001_2002'] is the same as gdf_nagrt_0001['NAGRT_0001']
print((df_final_agr['NAGRT_2001'] == gdf_nagrt_0001['NAGRT_0001']).all())

#### Merging NAGRT (2001-2022) with other features

In [ ]:
# copy  df_final_agr
df = df_final_agr.copy()

In [ ]:
# load geopackage with PM2.5, Burned areas an other features
gdf = gpd.read_file("pdt/asthma_mortality/data/gpkg/tma_pm25_ba_pd_pdpm25_2001_2022.gpkg")

In [ ]:
# Perform a left merge, preserving all rows from gdf
gdf_nagrt_0122 = gdf.merge(df, on='IDDPTO', how='left')

In [ ]:
# visualize gdf_nagrt_0122
init_notebook_mode(all_interactive=True)
gdf_nagrt_0122.head()

In [ ]:
gdf_nagrt_0122.shape

In [ ]:
# Save dataset with NAGRT (2001-2022) as other features as a gpkg file
gdf_nagrt_0122.to_file("pdt/asthma_mortality/data/gpkg/tma_pm25_ba_pd_pdpm25_agrt_2001_2022.gpkg", driver="GPKG")

## 📱Calculating the Normalized Natural Wooded Vegetation Transitions areas (NNWVT)

### Computing the NNWVT for the period 2000-2001

In this section, we will calculate the NNWVT areas for the period 2000–2001. This includes all areas that changed from natural wooded vegetation class to  any other land cover category between 2000 and 2001 (losses). The resulting dataset will be aggregated by department, normalized by each department’s surface area, and then multiplied by 1,000 km² to enhance interpretability.

In [ ]:
# Load the transition image and select the 2000–2001 band
transitions = ee.Image(
    'projects/mapbiomas-public/assets/argentina/collection1/mapbiomas_argentina_collection1_transitions_v1'
).select('transitions_2000_2001')

# Define the Natural Wooded Vegetation classes
source_classes = [3, 4, 45, 6]

# Extract from and to classes
from_class = transitions.divide(100).floor().toInt()
to_class = transitions.mod(100).toInt()

# Condition: from_class in source_classes
from_source = from_class.eq(source_classes[0])
for cls in source_classes[1:]:
    from_source = from_source.Or(from_class.eq(cls))

# Condition: to_class not in source_classes
to_not_source = to_class.neq(source_classes[0])
for cls in source_classes[1:]:
    to_not_source = to_not_source.And(to_class.neq(cls))

# Final mask: pixels that were natural vegetation but changed to something else (losses)
loss_mask = from_source.And(to_not_source)

# Binary mask image
binary_mask = loss_mask.selfMask()

# Load departments
departments = ee.FeatureCollection('projects/ee-pdt/assets/argentina/ar_dpto')

# Pixel area in km²
pixel_area_km2 = ee.Image.pixelArea().divide(1e6)

# Multiply mask by pixel area
loss_area_img = binary_mask.multiply(pixel_area_km2)

# Function to compute total loss area and normalize
def compute_loss(feature):
    area_loss = loss_area_img.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=feature.geometry(),
        scale=30,
        maxPixels=1e13
    ).get('transitions_2000_2001')

    dept_area_km2 = feature.geometry().area().divide(1e6)
    loss_km2 = ee.Number(area_loss)
    loss_norm = loss_km2.divide(dept_area_km2)

    # Normalize by 1000 km² and round to 2 decimals
    loss_norm_1000 = loss_norm.multiply(1000).multiply(100).round().divide(100)

    return feature.set({'NNWVT_0001': loss_norm_1000})

In [ ]:
# Apply to departments
result = departments.map(compute_loss)

In [ ]:
# Convert FeatureCollection into a GeoDataFrame
gdf_nnwvt_0001 = geemap.ee_to_gdf(result)

In [ ]:
# Display the first few rows of the DataFrame
init_notebook_mode(all_interactive=True)
gdf_nnwvt_0001

#### Mapping the NNWVT period 2000-2001

In this section we will display vector data (GeoDataFrame with NWVT_0001) using folium.Choropleth

In [ ]:
# Create base map centered on Argentina
m = folium.Map(location=[-38.4, -63.6], zoom_start=5, control_scale=True)

# Add Choropleth layer from gdf
choropleth = folium.Choropleth(
    geo_data=gdf_nnwvt_0001,
    name='NNWVT 2000-2001',
    data=gdf_nnwvt_0001.drop(columns=['geometry']),
    columns=['IDDPTO', 'NNWVT_0001'],
    key_on='feature.properties.IDDPTO',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.3,
    nan_fill_color='gray',
    legend_name='Normalized Non-wooden Vegetation transition (NNWVT) period 2000-2001'
)
choropleth.add_to(m)

# Now add an interactive GeoJson layer with popups
geojson = folium.GeoJson(
    gdf_nnwvt_0001,
    name='Interactive Layer',
    style_function=lambda feature: {
        'fillOpacity': 0,
        'color': 'black',
        'weight': 0.3,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['IDDPTO', 'NNWVT_0001'],
        aliases=['Dept ID:', 'Losses (/1000 km²):'],
        localize=True,
        sticky=True,
        labels=True
    )
).add_to(m)

# Define visualization parameters for binary_mask
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['#ffffff', '#000000']  # white = no change, red = loss
}

# Get map tiles from Earth Engine image
map_id_dict = binary_mask.getMapId(vis_params)

# Create Folium TileLayer
tile = folium.raster_layers.TileLayer(
    tiles=map_id_dict['tile_fetcher'].url_format,
    attr='Earth Engine',
    name='Mapbiomas transition Raster (Black)',
    overlay=True,
    control=True
)

# Add raster layer to folium map
m.add_child(tile)

# Add layer control
folium.LayerControl().add_to(m)

In [ ]:
# display the map
m

####  Checking results of NNWVT period 2000-2001

In this section, we will extract the transition image where binary_mask is equal to 1 for the department with the highest NNWVT (IDDPTO = "62007"). We will then verify whether pixel values correspond to transitions that start  in, but do not end from any Natural Wooden Vegetation class code (3, 4, 45, 6)

In [ ]:
# Select the feature with IDDPTO '62007'
dept_62007 = departments.filter(ee.Filter.eq('IDDPTO', '62007')).first()

# Clip the transitions image by the geometry of the selected department
transitions_clipped = transitions.clip(dept_62007.geometry())
binary_mask_clipped = binary_mask.clip(dept_62007.geometry())

# Get pixel values of the transitions image where binary_mask_clipped == 1
# Define the reducer to get a list of pixel values
reducer = ee.Reducer.toList()

# Reduce the clipped transitions image using the clipped binary mask
# We multiply the transitions image by the binary mask so only pixels where
# the mask is 1 are included in the reduction.
pixel_values = transitions_clipped.updateMask(binary_mask_clipped).reduceRegion(
    reducer=reducer,
    geometry=dept_62007.geometry(),
    scale=30,  # Use the same scale as before
    maxPixels=1e13
)

# Extract the list of pixel values
list_of_values = pixel_values.get('transitions_2000_2001').getInfo()

In [ ]:
# get unique values of list_of_values
unique_values = list(set(list_of_values))

In [ ]:
# print values divided by 100 (from class before period the dot, to clas after the dot)
# the result meets our checking condition
for value in unique_values:
    print(value/100)

###  Computing NNWVT from 2001 to 2022


In this section, we will calculate the NNWVT areas for the period 2001–2022. This includes all areas that changed from natural wooded vegetation class to  any other land cover category between 2001 and 2002 (losses). The resulting dataset will be aggregated by department, normalized by each department’s surface area, and then multiplied by 1,000 km² to enhance interpretability.

In [ ]:
# Department boundaries
departments = ee.FeatureCollection('projects/ee-pdt/assets/argentina/ar_dpto')

# Natural Wooded Vegetation source classes
source_classes = [3, 4, 45, 6]

# Pixel area in km²
pixel_area_km2 = ee.Image.pixelArea().divide(1e6)

In [ ]:
def process_transition(y1):

  """
  Compute the Normalized Non-wooden Vegetation transition (NNWVT)
  for a specific year-to-year transition and return it as a GeoDataFrame.

  Parameters:
      y1 (int): The start year of the transition (e.g., 2001 for 2001–2002)

  Returns:
      GeoDataFrame with columns: ['IDDPTO', 'NNWVT_y1_y2']

      Note: The name of the output column should be NNWVT_y1_y2,
            but for the sake of posterior analysis we choose NNWVT_y2 (y2 is the end year)
  """

  try:

    # Define transition band and output column name
    y2 = y1 + 1
    band_name = f'transitions_{y1}_{y2}'
    # col_name = f'NNWVT_{y1}_{y2}'
    col_name = f'NNWVT_{y2}'

    # Load transition band
    transitions = ee.Image('projects/mapbiomas-public/assets/argentina/collection1/mapbiomas_argentina_collection1_transitions_v1') \
        .select(band_name)

    # Extract from/to class codes
    from_class = transitions.divide(100).floor().toInt()
    to_class = transitions.mod(100).toInt()

    # from_class in source_classes
    from_source = from_class.eq(source_classes[0])
    for cls in source_classes[1:]:
        from_source = from_source.Or(from_class.eq(cls))

    # to_class not in source_classes
    to_not_source = to_class.neq(source_classes[0])
    for cls in source_classes[1:]:
        to_not_source = to_not_source.And(to_class.neq(cls))

    # Final mask: loss of native vegetation
    loss_mask = from_source.And(to_not_source)

    # Binary mask
    binary_mask = loss_mask.selfMask()

    # Compute area of loss
    loss_area_img = binary_mask.multiply(pixel_area_km2)

    # Compute loss by department
    def compute_loss(feature):
        area_loss = loss_area_img.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=feature.geometry(),
            scale=30,
            maxPixels=1e13
        ).get(band_name)

        dept_area_km2 = feature.geometry().area().divide(1e6)
        loss_km2 = ee.Number(area_loss)
        loss_norm = loss_km2.divide(dept_area_km2)
        loss_norm_1000 = loss_norm.multiply(1000).multiply(100).round().divide(100)

        return feature.set({col_name: loss_norm_1000})

    # Apply computation
    result = departments.map(compute_loss)

    # Convert to GeoDataFrame
    gdf = geemap.ee_to_gdf(result).drop(columns='geometry')

    # Return DataFrame with only ID and calculated column
    return gdf[['IDDPTO', col_name]]

  except Exception as e:
    print(f"[ERROR] Processing {y1}-{y2}: {e}")
    return pd.DataFrame(columns=['IDDPTO', f"NNWVT_{y1}_{y2}"])

In [ ]:
# list to store all dataframe derived from the for loop
all_dfs = []

In [ ]:
# Loop from 2000 to 2021 (ending at transitions_2021_2022)
for year in range(2000, 2022):
    print(f"Processing transition {year}-{year+1}")
    df = process_transition(year)
    all_dfs.append(df)

In [ ]:
# Merge all dataframes on 'IDDPTO'
df_final = all_dfs[0]
for df in all_dfs[1:]:
    df_final = df_final.merge(df, on='IDDPTO', how='outer')

In [ ]:
# visualize the data.frame
init_notebook_mode(all_interactive=True)
df_final.head()

In [ ]:
# get info of the data.frame
df_final.info()

In [ ]:
# test if df_final['NNWVT_2001'] is the same as gdf_nagrt_0001['NAGRT_00001']
print((df_final['NNWVT_2001'] == gdf_nnwvt_0001['NNWVT_0001']).all())

#### Merge NNWVT (2001-2022) with other features

In [ ]:
# load geopackage with PM2.5, Burned areas an other datasets
gdf = gpd.read_file("pdt/asthma_mortality/data/gpkg/tma_pm25_ba_pd_pdpm25_agrt_2001_2022.gpkg")

In [ ]:
# Perform a left merge, preserving all rows from gdf
gdf_nnwvt_0122 = gdf.merge(df_final, on='IDDPTO', how='left')

In [ ]:
# visualize gdf_nagrt_0122
init_notebook_mode(all_interactive=True)
gdf_nnwvt_0122.head()

In [ ]:
gdf_nnwvt_0122.shape

In [ ]:
# Save dataset with NAGRT (2001-2022) as other features as a gpkg file
gdf_nnwvt_0122.to_file("pdt/asthma_mortality/data/gpkg/tma_pm25_ba_pd_pdpm25_agrt_nwvt_2001_2022.gpkg", driver="GPKG")

## 🏘️ Calculating the Normalized Built-up Transitions areas (NBUT)

The GLAD dataset for Built-up classes is only available every 5 years from 2000 to 2020 at https://code.earthengine.google.com/f9f56ceb38ed9e911767c4014eeb536d. We will compute the NBUT areas between the available years using dummies transition values as explain bellow and the [Global Land Use and Land Cover 2000-2020 legend](https://storage.googleapis.com/earthenginepartners-hansen/GLCLU2000-2020/v2/legend.xlsx).

In [ ]:
# Example on how to get a dummy transition value
# From is the start year
# To is the End year
From = 66
To = 250
value = (From * 256) + To
print("Dummy transition value is:",  value)

In [ ]:
# get From and To value from dummie trasition value
From_2 = value//256
To_2 = value%256
print("Value:", value, "From:", From_2, "To:", To_2)

### Computing the NBUT for the period 2000-2005

We will calculate the NBUT areas for the period 2000–2005. This includes all areas that changed from any class to built-up category between 2000 and 2005 (gainings). The resulting dataset will be aggregated by department, normalized by each department’s surface area, and then multiplied by 1,000 km² to enhance interpretability.

$$NBUT = \left(\text{Built-up transition area}\right/\text{Dep.area})*1000$$

In [ ]:
# load argentina boundaries
ar_poly = ee.FeatureCollection('projects/ee-pdt/assets/argentina/ar_poly')

In [ ]:
# load glad OceanMask
landmask = ee.Image("projects/glad/OceanMask").lte(1)

In [ ]:
# load glad lulc 2000
m00 = ee.Image('projects/glad/GLCLU2020/v2/LCLUC_2000').updateMask(landmask)

# clip m00 to ar_poly
m00_ar = m00.clip(ar_poly)

# load glad lulc 2005
m05 = ee.Image('projects/glad/GLCLU2020/v2/LCLUC_2005').updateMask(landmask)

# clip m00 to ar_poly
m05_ar = m05.clip(ar_poly)

# create a transition image where each pixel value of m00 is multiplied by 256 and the sum the pixel value of m05
transition_0005_ar = m00_ar.multiply(256).add(m05_ar)

In [ ]:
# Define target built-up classes
target_classes = [250]

# Extract from and to classes
from_class = transition_0005_ar.divide(256).floor().toInt()
to_class = transition_0005_ar.mod(256).toInt()

# Condition: to_class is in target_classes
to_target = to_class.eq(target_classes[0])
for cls in target_classes[1:]:
    to_target = to_target.Or(to_class.eq(cls))

# Condition: from_class is NOT in target_classes
from_not_target = from_class.neq(target_classes[0])
for cls in target_classes[1:]:
    from_not_target = from_not_target.And(from_class.neq(cls))

# Final mask: pixels that transitioned to built-up from a different class
builtup_mask = to_target.And(from_not_target)

# Binary mask image
binary_mask = builtup_mask.selfMask()

# Pixel area in km²
pixel_area_km2 = ee.Image.pixelArea().divide(1e6)

# Multiply to get built-up area in km² per pixel
builtup_area_img = binary_mask.multiply(pixel_area_km2)

# Load departments
departments = ee.FeatureCollection('projects/ee-pdt/assets/argentina/ar_dpto')

# Function to compute expansion per department
def compute_builtup(feature):
    area_built = builtup_area_img.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=feature.geometry(),
        scale=30,
        maxPixels=1e13
    ).get('b1')  # band name

    dept_area_km2 = feature.geometry().area().divide(1e6)
    built_km2 = ee.Number(area_built)
    built_norm = built_km2.divide(dept_area_km2)

    # Normalize per 1000 km² and round to 2 decimals
    built_norm_1000 = built_norm.multiply(1000).multiply(100).round().divide(100)

     # The name of the feature should be NBUT_2000_2005,
     # but for the sake of posterior analysis we choose NBUT_2005
    return feature.set({'NBUT_2005': built_norm_1000})

# Map over all departments
result = departments.map(compute_builtup)


In [ ]:
# Convert to GeoDataFrame and extract desired columns
gdf_but_0005 = geemap.ee_to_gdf(result)
df_but_0005 = gdf_but_0005[['IDDPTO', 'NBUT_2005']]

In [ ]:
# visualize dataframe
init_notebook_mode(all_interactive=True)
df_but_0005

#### Mapping the NBUT period 2000-2005

In this section we will display vector data (GeoDataFrame with NBUT 2000-2005) using folium.Choropleth

In [ ]:
# Create base map centered on Argentina
m = folium.Map(location=[-38.4, -63.6], zoom_start=5, control_scale=True)

# m = folium.Map(
#     location=[-38.4, -63.6],
#     zoom_start=5,
#     control_scale=True,
#     tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
#     attr='Esri',
#     name='Esri Satellite',
#     overlay=False,
#     control=True
# )

# Add Choropleth layer from gdf
choropleth = folium.Choropleth(
    geo_data=gdf_but_0005,
    name='NBUT 2000-2005',
    data=df_but_0005,
    columns=['IDDPTO', 'NBUT_2005'],
    key_on='feature.properties.IDDPTO',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.3,
    nan_fill_color='gray',
    legend_name=' Normalized Built-up transitions areas period 2000-2005'
)
choropleth.add_to(m)

# Now add an interactive GeoJson layer with popups
geojson = folium.GeoJson(
    gdf_but_0005,
    name='Interactive Layer',
    style_function=lambda feature: {
        'fillOpacity': 0,
        'color': 'black',
        'weight': 0.3,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['IDDPTO', 'NBUT_2005'],
        aliases=['Dept ID:', 'Expansion (/1000 km²):'],
        localize=True,
        sticky=True,
        labels=True
    )
).add_to(m)

# Display expansion raster (binary_mask) as red overlay ---
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['#ffffff', '#000000']  # white = no change, red = expansion
}

# Get map tiles from Earth Engine image
map_id_dict = binary_mask.getMapId(vis_params)

# Create Folium TileLayer
tile = folium.raster_layers.TileLayer(
    tiles=map_id_dict['tile_fetcher'].url_format,
    attr='Earth Engine',
    name='GLAD transition Raster (Black)',
    overlay=True,
    control=True
)

# Add raster layer to folium map
m.add_child(tile)

# Add layer control
folium.LayerControl().add_to(m)

In [ ]:
# display map
m

####  Checking results of NBUT period 2000-2005

In this section, we will extract the transition image where binary_mask is equal to 1 for the department with the highest NBUT (IDDPTO = "02000"). We will then verify whether pixel values correspond to transitions that end in, but do not start from any built-up class code (250, 251, 252, 253)

In [ ]:
# Select the feature with IDDPTO '06134'
dept_02000 = departments.filter(ee.Filter.eq('IDDPTO', '02000')).first()

# Clip the transitions image by the geometry of the selected department
transitions_clipped = transition_0005_ar.clip(dept_02000.geometry())
binary_mask_clipped = binary_mask.clip(dept_02000.geometry())

# Get pixel values of the transitions image where binary_mask_clipped == 1
# Define the reducer to get a list of pixel values
reducer = ee.Reducer.toList()

# Reduce the clipped transitions image using the clipped binary mask
# We multiply the transitions image by the binary mask so only pixels where
# the mask is 1 are included in the reduction.
pixel_values = transitions_clipped.updateMask(binary_mask_clipped).reduceRegion(
    reducer=reducer,
    geometry=dept_02000.geometry(),
    scale=30,  # Use the same scale as before
    maxPixels=1e13
)

# Extract the list of pixel values
list_of_values = pixel_values.get('b1').getInfo()

In [ ]:
# get unique values of list_of_values
unique_values = list(set(list_of_values))

In [ ]:
# list of values from classes
lovf = []
# verify the "from" class is not in target_classes
target_classes = [250, 251, 252, 253]
for value in unique_values:
    if value // 256 in target_classes:
        print(value)
    else:
        lovf.append(value)
        if len(lovf) == len(unique_values):
            print("No value of 'from' class is in target_classes")


In [ ]:
# list of values to classes
lovt = []
# verify the "to" class is in target_classes
target_classes = [250, 251, 252, 253]
for value in unique_values:
    if value % 256 in target_classes:
        lovt.append(value)
        if len(lovt) == len(unique_values):
            print("All values of 'to' class are in target_classes")
    else:
        print(value)


#### Imputing values NBUT between 2001 and 2005

For the time series analysis, we require NBUT data for each year between 2001 and 2005. Since no significant built-up gains are expected during this period, we will assume stability and impute the 2005 value backward from 2001 to 2004.



In [ ]:
# impute missing years
df_but_0005['NBUT_2001'] = df_but_0005['NBUT_2005']
df_but_0005['NBUT_2002'] = df_but_0005['NBUT_2005']
df_but_0005['NBUT_2003'] = df_but_0005['NBUT_2005']
df_but_0005['NBUT_2004'] = df_but_0005['NBUT_2005']

In [ ]:
# rearrange colums
df_but_0005 = df_but_0005[['IDDPTO', 'NBUT_2001', 'NBUT_2002', 'NBUT_2003', 'NBUT_2004', 'NBUT_2005']]

In [ ]:
# visualize data.frame
init_notebook_mode(all_interactive=True)
df_but_0005

#### Merge NBUT (2001-2005) with other features

In [ ]:
# load geopackage with PM2.5, Burned areas an other datasets
gdf = gpd.read_file("pdt/asthma_mortality/data/gpkg/tma_pm25_ba_pd_pdpm25_agrt_nwvt_2001_2022.gpkg")

In [ ]:
# Perform a left merge, preserving all rows from gdf
gdf_nbut_0005 = gdf.merge(df_but_0005, on='IDDPTO', how='left')

In [ ]:
# visualize gdf
init_notebook_mode(all_interactive=True)
gdf_nbut_0005.head()

In [ ]:
gdf_nbut_0005.shape

In [ ]:
# Save dataset with NBUT (2000-2005) as other features as a gpkg file
gdf_nbut_0005.to_file("pdt/asthma_mortality/data/gpkg/tma_pm25_ba_pd_pdpm25_agrt_nwvt_nbut0005_2001_2022.gpkg", driver="GPKG")

### Computing the NBUT for the period 2005-2010

In this section, we will calculate the NBUT areas for the period 2005–2010. This includes all areas that changed from any class to built-up category between 2005 and 2010 (gainings). The resulting dataset will be aggregated by department, normalized by each department’s surface area, and then multiplied by 1,000 km² to enhance interpretability.

In [ ]:
# load argentina boundaries
ar_poly = ee.FeatureCollection('projects/ee-pdt/assets/argentina/ar_poly')

In [ ]:
# load glad OceanMask
landmask = ee.Image("projects/glad/OceanMask").lte(1)

In [ ]:
# load glad lulc 2005
m05 = ee.Image('projects/glad/GLCLU2020/v2/LCLUC_2005').updateMask(landmask)

# clip m05 to ar_poly
m05_ar = m05.clip(ar_poly)

# load glad lulc 2010
m10 = ee.Image('projects/glad/GLCLU2020/v2/LCLUC_2010').updateMask(landmask)

# clip m10 to ar_poly
m10_ar = m10.clip(ar_poly)

# create a transition image where each pixel value of m05 is multiplied by 256 and the sum the pixel value of m10
transition_0510_ar = m05_ar.multiply(256).add(m10_ar)

In [ ]:
# Define target built-up class
target_classes = [250]

# Extract from and to classes
from_class = transition_0510_ar.divide(256).floor().toInt()
to_class = transition_0510_ar.mod(256).toInt()

# Condition: to_class is in target_classes
to_target = to_class.eq(target_classes[0])
for cls in target_classes[1:]:
    to_target = to_target.Or(to_class.eq(cls))

# Condition: from_class is NOT in target_classes
from_not_target = from_class.neq(target_classes[0])
for cls in target_classes[1:]:
    from_not_target = from_not_target.And(from_class.neq(cls))

# Final mask: pixels that transitioned to built-up from a different class
builtup_mask = to_target.And(from_not_target)

# Binary mask image
binary_mask = builtup_mask.selfMask()

# Pixel area in km²
pixel_area_km2 = ee.Image.pixelArea().divide(1e6)

# Multiply to get built-up area in km² per pixel
builtup_area_img = binary_mask.multiply(pixel_area_km2)

# Load departments
departments = ee.FeatureCollection('projects/ee-pdt/assets/argentina/ar_dpto')

# Function to compute expansion per department
def compute_builtup(feature):
    area_built = builtup_area_img.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=feature.geometry(),
        scale=30,
        maxPixels=1e13
    ).get('b1')  # band name

    dept_area_km2 = feature.geometry().area().divide(1e6)
    built_km2 = ee.Number(area_built)
    built_norm = built_km2.divide(dept_area_km2)

    # Normalize per 1000 km² and round to 2 decimals
    built_norm_1000 = built_norm.multiply(1000).multiply(100).round().divide(100)

    # The name of the feature should be NBUT_2005_2010,
    # but for the sake of posterior analysis we choose NBUT_2010
    return feature.set({'NBUT_2010': built_norm_1000})

# Map over all departments
result = departments.map(compute_builtup)

In [ ]:
# Convert to GeoDataFrame and extract desired columns
gdf_but_0510 = geemap.ee_to_gdf(result)
df_but_0510 = gdf_but_0510[['IDDPTO', 'NBUT_2010']]

In [ ]:
# visualize dataframe
init_notebook_mode(all_interactive=True)
df_but_0510

#### Mapping the NBUT period 2005-2010

In this section we will display vector data (GeoDataFrame with NBUT 2005-2010) using folium.Choropleth

In [ ]:
# Create base map centered on Argentina
m = folium.Map(location=[-38.4, -63.6], zoom_start=5, control_scale=True)

# Add Choropleth layer from gdf
choropleth = folium.Choropleth(
    geo_data=gdf_but_0510,
    name='NBUT 2005-2010',
    data=df_but_0510,
    columns=['IDDPTO', 'NBUT_2010'],
    key_on='feature.properties.IDDPTO',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.3,
    nan_fill_color='gray',
    legend_name=' Normalized Built-up transitions areas period 2005-2010'
)
choropleth.add_to(m)

# Now add an interactive GeoJson layer with popups
geojson = folium.GeoJson(
    gdf_but_0510,
    name='Interactive Layer',
    style_function=lambda feature: {
        'fillOpacity': 0,
        'color': 'black',
        'weight': 0.3,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['IDDPTO', 'NBUT_2010'],
        aliases=['Dept ID:', 'Expansion (/1000 km²):'],
        localize=True,
        sticky=True,
        labels=True
    )
).add_to(m)

# Display expansion raster (binary_mask) as red overlay ---
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['#ffffff', '#000000']  # white = no change, red = expansion
}

# Get map tiles from Earth Engine image
map_id_dict = binary_mask.getMapId(vis_params)

# Create Folium TileLayer
tile = folium.raster_layers.TileLayer(
    tiles=map_id_dict['tile_fetcher'].url_format,
    attr='Earth Engine',
    name='GLAD transition Raster (Black)',
    overlay=True,
    control=True
)

# Add raster layer to folium map
m.add_child(tile)

# Add layer control
folium.LayerControl().add_to(m)

In [ ]:
# display map
m

####  Checking results of NBUT period 2005-2010

In this section, we will extract the transition image where binary_mask is equal to 1 for the department with the highest NBUT (IDDPTO = "50007"). We will then verify whether pixel values correspond to transitions that end in, but do not start from any built-up class code (250, 251, 252, 253)

In [ ]:
# Select the feature with IDDPTO '50007'
dept_50007 = departments.filter(ee.Filter.eq('IDDPTO', '50007')).first()

# Clip the transitions image by the geometry of the selected department
transitions_clipped = transition_0510_ar.clip(dept_50007.geometry())
binary_mask_clipped = binary_mask.clip(dept_50007.geometry())

# Get pixel values of the transitions image where binary_mask_clipped == 1
# Define the reducer to get a list of pixel values
reducer = ee.Reducer.toList()

# Reduce the clipped transitions image using the clipped binary mask
# We multiply the transitions image by the binary mask so only pixels where
# the mask is 1 are included in the reduction.
pixel_values = transitions_clipped.updateMask(binary_mask_clipped).reduceRegion(
    reducer=reducer,
    geometry=dept_50007.geometry(),
    scale=30,  # Use the same scale as before
    maxPixels=1e13
)

# Extract the list of pixel values
list_of_values = pixel_values.get('b1').getInfo()

In [ ]:
# get unique values of list_of_values
unique_values = list(set(list_of_values))

In [ ]:
# list of values from classes
lovf = []
# verify the "from" class is not in target_classes
target_classes = [250, 251, 252, 253]
for value in unique_values:
    if value // 256 in target_classes:
        print(value)
    else:
        lovf.append(value)
        if len(lovf) == len(unique_values):
            print("No value of 'from' class is in target_classes")


In [ ]:
# list of values to classes
lovt = []
# verify the "to" class is in target_classes
target_classes = [250, 251, 252, 253]
for value in unique_values:
    if value % 256 in target_classes:
        lovt.append(value)
        if len(lovt) == len(unique_values):
            print("All values of 'to' class are in target_classes")
    else:
        print(value)


#### Imputing values betwwen 2005 and 2010

For the time series analysis, we require NBUT data for each year between 2005 and 2010. Since no significant built-up gains are expected during this period, we will assume stability and impute the 2010 value backward from 2006 to 2009



In [ ]:
# impute missing years
df_but_0510['NBUT_2006'] = df_but_0510['NBUT_2010']
df_but_0510['NBUT_2007'] = df_but_0510['NBUT_2010']
df_but_0510['NBUT_2008'] = df_but_0510['NBUT_2010']
df_but_0510['NBUT_2009'] = df_but_0510['NBUT_2010']

In [ ]:
# rearrange colums
df_but_0510 = df_but_0510[['IDDPTO', 'NBUT_2006', 'NBUT_2007', 'NBUT_2008', 'NBUT_2009', 'NBUT_2010']]

In [ ]:
# visualize data.frame
init_notebook_mode(all_interactive=True)
df_but_0510

#### Merge NBUT (2005-2010) with other features

In [ ]:
# load geopackage with PM2.5, Burned areas an other datasets
gdf = gpd.read_file("pdt/asthma_mortality/data/gpkg/tma_pm25_ba_pd_pdpm25_agrt_nwvt_nbut0005_2001_2022.gpkg")

In [ ]:
# Perform a left merge, preserving all rows from gdf
gdf_nbut_0510 = gdf.merge(df_but_0510, on='IDDPTO', how='left')

In [ ]:
# visualize gdf
init_notebook_mode(all_interactive=True)
gdf_nbut_0510.head()

In [ ]:
gdf_nbut_0510.shape

In [ ]:
# Save dataset with NBUT (2005-2010) as other features as a gpkg file
gdf_nbut_0510.to_file("pdt/asthma_mortality/data/gpkg/tma_pm25_ba_pd_pdpm25_agrt_nwvt_nbut0010_2001_2022.gpkg", driver="GPKG")

### Computing the NBUT for the period 2010-2015

In this section, we will calculate the NBUT areas for the period 2010–2015. This includes all areas that changed from any class to built-up category between 2010 and 2015 (gainings). The resulting dataset will be aggregated by department, normalized by each department’s surface area, and then multiplied by 1,000 km² to enhance interpretability.

In [ ]:
# load argentina boundaries
ar_poly = ee.FeatureCollection('projects/ee-pdt/assets/argentina/ar_poly')

In [ ]:
# load glad OceanMask
landmask = ee.Image("projects/glad/OceanMask").lte(1)

In [ ]:
# load glad lulc 2010
m10 = ee.Image('projects/glad/GLCLU2020/v2/LCLUC_2010').updateMask(landmask)

# clip m10 to ar_poly
m10_ar = m10.clip(ar_poly)

# load glad lulc 2015
m15 = ee.Image('projects/glad/GLCLU2020/v2/LCLUC_2015').updateMask(landmask)

# clip m15 to ar_poly
m15_ar = m15.clip(ar_poly)

# create a transition image where each pixel value of m10 is multiplied by 256 and the sum the pixel value of m15
transition_1015_ar = m10_ar.multiply(256).add(m15_ar)

In [ ]:
# Define target built-up classes
target_classes = [250]

# Extract from and to classes
from_class = transition_1015_ar.divide(256).floor().toInt()
to_class = transition_1015_ar.mod(256).toInt()

# Condition: to_class is in target_classes
to_target = to_class.eq(target_classes[0])
for cls in target_classes[1:]:
    to_target = to_target.Or(to_class.eq(cls))

# Condition: from_class is NOT in target_classes
from_not_target = from_class.neq(target_classes[0])
for cls in target_classes[1:]:
    from_not_target = from_not_target.And(from_class.neq(cls))

# Final mask: pixels that transitioned to built-up from a different class
builtup_mask = to_target.And(from_not_target)

# Binary mask image
binary_mask = builtup_mask.selfMask()

# Pixel area in km²
pixel_area_km2 = ee.Image.pixelArea().divide(1e6)

# Multiply to get built-up area in km² per pixel
builtup_area_img = binary_mask.multiply(pixel_area_km2)

# Load departments
departments = ee.FeatureCollection('projects/ee-pdt/assets/argentina/ar_dpto')

# Function to compute expansion per department
def compute_builtup(feature):
    area_built = builtup_area_img.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=feature.geometry(),
        scale=30,
        maxPixels=1e13
    ).get('b1')  # band name

    dept_area_km2 = feature.geometry().area().divide(1e6)
    built_km2 = ee.Number(area_built)
    built_norm = built_km2.divide(dept_area_km2)

    # Normalize per 1000 km² and round to 2 decimals
    built_norm_1000 = built_norm.multiply(1000).multiply(100).round().divide(100)

    # The name of the feature should be NBUT_2010_2015,
    # but for the sake of posterior analysis we choose NBUT_2015
    return feature.set({'NBUT_2015': built_norm_1000})

# Map over all departments
result = departments.map(compute_builtup)

In [ ]:
# Convert to GeoDataFrame and extract desired columns
gdf_but_1015 = geemap.ee_to_gdf(result)
df_but_1015 = gdf_but_1015[['IDDPTO', 'NBUT_2015']]

In [ ]:
# visualize dataframe
init_notebook_mode(all_interactive=True)
df_but_1015

#### Mapping the NBUT period 2010-2015

In this section we will display vector data (GeoDataFrame with NBUT 2010-2015) using folium.Choropleth

In [ ]:
# Create base map centered on Argentina
m = folium.Map(location=[-38.4, -63.6], zoom_start=5, control_scale=True)

# m = folium.Map(
#     location=[-38.4, -63.6],
#     zoom_start=5,
#     control_scale=True,
#     tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
#     attr='Esri',
#     name='Esri Satellite',
#     overlay=False,
#     control=True
# )

# Add Choropleth layer from gdf
choropleth = folium.Choropleth(
    geo_data=gdf_but_1015,
    name='NBUT 2010-2015',
    data=df_but_1015,
    columns=['IDDPTO', 'NBUT_2015'],
    key_on='feature.properties.IDDPTO',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.3,
    nan_fill_color='gray',
    legend_name=' Normalized Built-up transitions areas period 2010-2015'
)
choropleth.add_to(m)

# Now add an interactive GeoJson layer with popups
geojson = folium.GeoJson(
    gdf_but_1015,
    name='Interactive Layer',
    style_function=lambda feature: {
        'fillOpacity': 0,
        'color': 'black',
        'weight': 0.3,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['IDDPTO', 'NBUT_2015'],
        aliases=['Dept ID:', 'Expansion (/1000 km²):'],
        localize=True,
        sticky=True,
        labels=True
    )
).add_to(m)

# Display expansion raster (binary_mask) as red overlay ---
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['#ffffff', '#000000']  # white = no change, red = expansion
}

# Get map tiles from Earth Engine image
map_id_dict = binary_mask.getMapId(vis_params)

# Create Folium TileLayer
tile = folium.raster_layers.TileLayer(
    tiles=map_id_dict['tile_fetcher'].url_format,
    attr='Earth Engine',
    name='GLAD transition Raster (Black)',
    overlay=True,
    control=True
)

# Add raster layer to folium map
m.add_child(tile)

# Add layer control
folium.LayerControl().add_to(m)

In [ ]:
# display map
m

####  Checking results of NBUT period 2010-2015

In this section, we will extract the transition image where binary_mask is equal to 1 for the department with the highest NBUT (IDDPTO = "50028"). We will then verify whether pixel values correspond to transitions that end in, but do not start from any built-up class code (250, 251, 252, 253)

In [ ]:
# Select the feature with IDDPTO '50028'
dept_50028 = departments.filter(ee.Filter.eq('IDDPTO', '50028')).first()

# Clip the transitions image by the geometry of the selected department
transitions_clipped = transition_1015_ar.clip(dept_50028.geometry())
binary_mask_clipped = binary_mask.clip(dept_50028.geometry())

# Get pixel values of the transitions image where binary_mask_clipped == 1
# Define the reducer to get a list of pixel values
reducer = ee.Reducer.toList()

# Reduce the clipped transitions image using the clipped binary mask
# We multiply the transitions image by the binary mask so only pixels where
# the mask is 1 are included in the reduction.
pixel_values = transitions_clipped.updateMask(binary_mask_clipped).reduceRegion(
    reducer=reducer,
    geometry=dept_50028.geometry(),
    scale=30,  # Use the same scale as before
    maxPixels=1e13
)

# Extract the list of pixel values
list_of_values = pixel_values.get('b1').getInfo()

In [ ]:
# get unique values of list_of_values
unique_values = list(set(list_of_values))

In [ ]:
# list of values from classes
lovf = []
# verify the "from" class is not in target_classes
target_classes = [250, 251, 252, 253]
for value in unique_values:
    if value // 256 in target_classes:
        print(value)
    else:
        lovf.append(value)
        if len(lovf) == len(unique_values):
            print("No value of 'from' class is in target_classes")


In [ ]:
# list of values to classes
lovt = []
# verify the "to" class is in target_classes
target_classes = [250, 251, 252, 253]
for value in unique_values:
    if value % 256 in target_classes:
        lovt.append(value)
        if len(lovt) == len(unique_values):
            print("All values of 'to' class are in target_classes")
    else:
        print(value)


#### Imputing values betwwen 2010 and 2015

For the time series analysis, we require NBUT data for each year between 2010 and 2015. Since no significant built-up gains are expected during this period, we will assume stability and impute the 2015 value backward from 2011 to 2014



In [ ]:
# impute missing years
df_but_1015['NBUT_2011'] = df_but_1015['NBUT_2015']
df_but_1015['NBUT_2012'] = df_but_1015['NBUT_2015']
df_but_1015['NBUT_2013'] = df_but_1015['NBUT_2015']
df_but_1015['NBUT_2014'] = df_but_1015['NBUT_2015']

In [ ]:
# rearrange colums
df_but_1015 = df_but_1015[['IDDPTO', 'NBUT_2011', 'NBUT_2012', 'NBUT_2013', 'NBUT_2014', 'NBUT_2015']]

In [ ]:
# visualize data.frame
init_notebook_mode(all_interactive=True)
df_but_1015

#### Merge NBUT (2010-2015) with other features

In [ ]:
# load geopackage with PM2.5, Burned areas an other datasets
gdf = gpd.read_file("pdt/asthma_mortality/data/gpkg/tma_pm25_ba_pd_pdpm25_agrt_nwvt_nbut0010_2001_2022.gpkg")

In [ ]:
# Perform a left merge, preserving all rows from gdf
gdf_nbut_1015 = gdf.merge(df_but_1015, on='IDDPTO', how='left')

In [ ]:
# visualize gdf
init_notebook_mode(all_interactive=True)
gdf_nbut_1015.head()

In [ ]:
gdf_nbut_1015.shape

In [ ]:
# Save dataset with NBUT (2005-2010) as other features as a gpkg file
gdf_nbut_1015.to_file("pdt/asthma_mortality/data/gpkg/tma_pm25_ba_pd_pdpm25_agrt_nwvt_nbut0015_2001_2022.gpkg", driver="GPKG")

### Computing the NBUT for the period 2015-2020

In this section, we will calculate the NBUT areas for the period 2015–2020. This includes all areas that changed from any class to built-up category between 2015 and 2020 (gainings). The resulting dataset will be aggregated by department, normalized by each department’s surface area, and then multiplied by 1,000 km² to enhance interpretability.

In [ ]:
# load argentina boundaries
ar_poly = ee.FeatureCollection('projects/ee-pdt/assets/argentina/ar_poly')

In [ ]:
# load glad OceanMask
landmask = ee.Image("projects/glad/OceanMask").lte(1)

In [ ]:
# load glad lulc 2015
m15 = ee.Image('projects/glad/GLCLU2020/v2/LCLUC_2015').updateMask(landmask)

# clip m15 to ar_poly
m15_ar = m15.clip(ar_poly)

# load glad lulc 2020
m20 = ee.Image('projects/glad/GLCLU2020/v2/LCLUC_2020').updateMask(landmask)

# clip m20 to ar_poly
m20_ar = m20.clip(ar_poly)

# create a transition image where each pixel value of m15 is multiplied by 256 and the sum the pixel value of m20
transition_1520_ar = m15_ar.multiply(256).add(m20_ar)

In [ ]:
# Define target built-up classes
target_classes = [250]

# Extract from and to classes
from_class = transition_1520_ar.divide(256).floor().toInt()
to_class = transition_1520_ar.mod(256).toInt()

# Condition: to_class is in target_classes
to_target = to_class.eq(target_classes[0])
for cls in target_classes[1:]:
    to_target = to_target.Or(to_class.eq(cls))

# Condition: from_class is NOT in target_classes
from_not_target = from_class.neq(target_classes[0])
for cls in target_classes[1:]:
    from_not_target = from_not_target.And(from_class.neq(cls))

# Final mask: pixels that transitioned to built-up from a different class
builtup_mask = to_target.And(from_not_target)

# Binary mask image
binary_mask = builtup_mask.selfMask()

# Pixel area in km²
pixel_area_km2 = ee.Image.pixelArea().divide(1e6)

# Multiply to get built-up area in km² per pixel
builtup_area_img = binary_mask.multiply(pixel_area_km2)

# Load departments
departments = ee.FeatureCollection('projects/ee-pdt/assets/argentina/ar_dpto')

# Function to compute expansion per department
def compute_builtup(feature):
    area_built = builtup_area_img.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=feature.geometry(),
        scale=30,
        maxPixels=1e13
    ).get('b1')  # band name

    dept_area_km2 = feature.geometry().area().divide(1e6)
    built_km2 = ee.Number(area_built)
    built_norm = built_km2.divide(dept_area_km2)

    # Normalize per 1000 km² and round to 2 decimals
    built_norm_1000 = built_norm.multiply(1000).multiply(100).round().divide(100)

    # The name of the feature should be NBUT_2015_2020,
    # but for the sake of posterior analysis we choose NBUT_2020
    return feature.set({'NBUT_2020': built_norm_1000})

# Map over all departments
result = departments.map(compute_builtup)

In [ ]:
# Convert to GeoDataFrame and extract desired columns
gdf_but_1520 = geemap.ee_to_gdf(result)
df_but_1520 = gdf_but_1520[['IDDPTO', 'NBUT_2020']]

In [ ]:
# visualize dataframe
init_notebook_mode(all_interactive=True)
df_but_1520

#### Mapping the NBUT period 2015-2020

In this section we will display vector data (GeoDataFrame with NBUT 2015-2020) using folium.Choropleth

In [ ]:
# Create base map centered on Argentina
m = folium.Map(location=[-38.4, -63.6], zoom_start=5, control_scale=True)

# m = folium.Map(
#     location=[-38.4, -63.6],
#     zoom_start=5,
#     control_scale=True,
#     tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
#     attr='Esri',
#     name='Esri Satellite',
#     overlay=False,
#     control=True
# )

# Add Choropleth layer from gdf
choropleth = folium.Choropleth(
    geo_data=gdf_but_1520,
    name='NBUT 2015-2020',
    data=df_but_1520,
    columns=['IDDPTO', 'NBUT_2020'],
    key_on='feature.properties.IDDPTO',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.3,
    nan_fill_color='gray',
    legend_name=' Normalized Built-up transitions areas period 2015-2020'
)
choropleth.add_to(m)

# Now add an interactive GeoJson layer with popups
geojson = folium.GeoJson(
    gdf_but_1520,
    name='Interactive Layer',
    style_function=lambda feature: {
        'fillOpacity': 0,
        'color': 'black',
        'weight': 0.3,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['IDDPTO', 'NBUT_2020'],
        aliases=['Dept ID:', 'Expansion (/1000 km²):'],
        localize=True,
        sticky=True,
        labels=True
    )
).add_to(m)

# Display expansion raster (binary_mask) as red overlay ---
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['#ffffff', '#000000']  # white = no change, red = expansion
}

# Get map tiles from Earth Engine image
map_id_dict = binary_mask.getMapId(vis_params)

# Create Folium TileLayer
tile = folium.raster_layers.TileLayer(
    tiles=map_id_dict['tile_fetcher'].url_format,
    attr='Earth Engine',
    name='GLAD transition Raster (Black)',
    overlay=True,
    control=True
)

# Add raster layer to folium map
m.add_child(tile)

# Add layer control
folium.LayerControl().add_to(m)

In [ ]:
# display map
m

####  Checking results of NBUT period 2015-2020

In this section, we will extract the transition image where binary_mask is equal to 1 for the department with the highest NBUT (IDDPTO = "06247"). We will then verify whether pixel values correspond to transitions that end in, but do not start from any built-up class code (250, 251, 252, 253)

In [ ]:
# Select the feature with IDDPTO '06274'
dept_06274 = departments.filter(ee.Filter.eq('IDDPTO', '06274')).first()

# Clip the transitions image by the geometry of the selected department
transitions_clipped = transition_1520_ar.clip(dept_06274.geometry())
binary_mask_clipped = binary_mask.clip(dept_06274.geometry())

# Get pixel values of the transitions image where binary_mask_clipped == 1
# Define the reducer to get a list of pixel values
reducer = ee.Reducer.toList()

# Reduce the clipped transitions image using the clipped binary mask
# We multiply the transitions image by the binary mask so only pixels where
# the mask is 1 are included in the reduction.
pixel_values = transitions_clipped.updateMask(binary_mask_clipped).reduceRegion(
    reducer=reducer,
    geometry=dept_06274.geometry(),
    scale=30,  # Use the same scale as before
    maxPixels=1e13
)

# Extract the list of pixel values
list_of_values = pixel_values.get('b1').getInfo()

In [ ]:
# get unique values of list_of_values
unique_values = list(set(list_of_values))

In [ ]:
# list of values from classes
lovf = []
# verify the "from" class is not in target_classes
target_classes = [250, 251, 252, 253]
for value in unique_values:
    if value // 256 in target_classes:
        print(value)
    else:
        lovf.append(value)
        if len(lovf) == len(unique_values):
            print("No value of 'from' class is in target_classes")


In [ ]:
# list of values to classes
lovt = []
# verify the "to" class is in target_classes
target_classes = [250, 251, 252, 253]
for value in unique_values:
    if value % 256 in target_classes:
        lovt.append(value)
        if len(lovt) == len(unique_values):
            print("All values of 'to' class are in target_classes")
    else:
        print(value)


#### Imputing values betwwen 2015 and 2020

For the time series analysis, we require NBUT data for each year between 2015 and 2020. Since no significant built-up gains are expected during this period, we will assume stability and impute the 2020 value backward from 2016 to 2019, and forward to 2021



In [ ]:
# impute missing years
df_but_1520['NBUT_2016'] = df_but_1520['NBUT_2020']
df_but_1520['NBUT_2017'] = df_but_1520['NBUT_2020']
df_but_1520['NBUT_2018'] = df_but_1520['NBUT_2020']
df_but_1520['NBUT_2019'] = df_but_1520['NBUT_2020']
df_but_1520['NBUT_2021'] = df_but_1520['NBUT_2020']

In [ ]:
# rearrange colums
df_but_1520 = df_but_1520[['IDDPTO', 'NBUT_2016', 'NBUT_2017', 'NBUT_2018', 'NBUT_2019', 'NBUT_2020', 'NBUT_2021']]

In [ ]:
# visualize data.frame
init_notebook_mode(all_interactive=True)
df_but_1520

#### Merge NBUT (2015-2020) with other features

In [ ]:
# load geopackage with PM2.5, Burned areas an other datasets
gdf = gpd.read_file("pdt/asthma_mortality/data/gpkg/tma_pm25_ba_pd_pdpm25_agrt_nwvt_nbut0015_2001_2022.gpkg")

In [ ]:
# Perform a left merge, preserving all rows from gdf
gdf_nbut_1520 = gdf.merge(df_but_1520, on='IDDPTO', how='left')

In [ ]:
# visualize gdf
init_notebook_mode(all_interactive=True)
gdf_nbut_1520.head()

In [ ]:
gdf_nbut_1520.shape

In [ ]:
# Save dataset with NBUT (2005-2010) as other features as a gpkg file
gdf_nbut_1520.to_file("pdt/asthma_mortality/data/gpkg/tma_pm25_ba_pd_pdpm25_agrt_nwvt_nbut0020_2001_2022.gpkg", driver="GPKG")

### Computing the NBUT for the period 2021-2022

In this section, we will calculate the NBUT (Normalized Built-Up Transitions) areas for the period 2021–2022. We will use the [Dynamic World V1](https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_DYNAMICWORLD_V1) (DW) availabe on GEE with 10m spatial resolution. This calculation includes all areas that transitioned from any land cover class to the built-up category during that time frame (i.e., gains). The resulting dataset will be aggregated by department, normalized by each department’s surface area, and multiplied by 1,000 km² to improve interpretability.

#### Calculating the Built-up Transition Image (2021-2022)

The size of the Dynamic World (DW) dataset exceeds the computational limits of a free Google Earth Engine (GEE) account. Therefore, we will begin by calculating the built-up transition image for the period 2021–2022. This image will be stored in our GEE Assets and subsequently used to compute the New Built-Up Transitions (NBUT) during this period.

We will mask built-up areas and assign a pixel value of 1 to generate a binary built-up dominant image for each year. Then, using predefined dummy transition values (as explained below), we will compute the New Built-Up Transition (NBUT) areas for the period 2001 to 2022.

In [ ]:
# Example on how to get a dummy transition value
# From is the start year
# To is the End year
From = 0 # non-build-up
To = 1 #  built-up
value = (From * 2) + To
print("Dummy transition value is:",  value)

In [ ]:
# get From and To value from dummie trasition value
From_2 = value//2
To_2 = value%2
print("Value:", value, "From:", From_2, "To:", To_2)

In [ ]:
# Load Argentina boundary
argentina = ee.FeatureCollection("projects/ee-pdt/assets/argentina/ar_poly")

# Define date ranges
start_date1 = '2021-01-01'
end_date1 = '2021-12-31'
start_date2 = '2022-01-01'
end_date2 = '2022-12-31'

# Load and clip Dynamic World mean images
dw_2021 = (
    ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")
    .filterDate(start_date1, end_date1)
    .max()
    .clip(argentina)
)

dw_2022 = (
    ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")
    .filterDate(start_date2, end_date2)
    .max()
    .clip(argentina)
)

# List of class bands in Dynamic World
prob_bands = [
    'water', 'trees', 'grass', 'flooded_vegetation', 'crops',
    'shrub_and_scrub', 'built', 'bare', 'snow_and_ice'
]

# Built-dominant classification for 2021
prob_array_2021 = dw_2021.select(prob_bands).toArray()
max_prob_index_2021 = prob_array_2021.arrayArgmax().arrayGet([0])
built_dominant_mask_2021 = max_prob_index_2021.eq(6)
built_dominant_bin_2021 = built_dominant_mask_2021.rename('built_2021').unmask(0).uint8()

# Built-dominant classification for 2022
prob_array_2022 = dw_2022.select(prob_bands).toArray()
max_prob_index_2022 = prob_array_2022.arrayArgmax().arrayGet([0])
built_dominant_mask_2022 = max_prob_index_2022.eq(6)
built_dominant_2022_bin = built_dominant_mask_2022.rename('built_2022').unmask(0).uint8()

# Detect transitions from non-built to built
built_2021_times2 = built_dominant_bin_2021.multiply(2)
built_sum = built_2021_times2.add(built_dominant_2022_bin)
built_up_transition_2021_2022 = built_sum.eq(1).rename('built_up_transition_2021_2022').uint8()

# Mask pixels where transition = 1
built_up_transition_masked = built_up_transition_2021_2022.updateMask(built_up_transition_2021_2022)

It is not necessary to export the `built_up_transition_masked` image as an asset, since it has already been created and shared. However, if you wish to export it yourself, you can uncomment the code below and execute it.



In [ ]:
# Define export parameters
# export_task = ee.batch.Export.image.toAsset(
#     image=built_up_transition_masked,
#     description='export_built_up_transition_masked 2021-2022',
#     assetId='projects/ee-pdt/assets/built_up_transition_2021_2022',
#     region=argentina.geometry(),
#     scale=10,
#     crs='EPSG:4326',
#     maxPixels=1e13
# )

In [ ]:
# Start the task
# export_task.start()
# print("Export task started. Check the Tasks tab in the Earth Engine Code Editor.")

#### Computing the NBUT in Cordoba Province (2020-2022)

Due to computational constraints, this analysis will be performed aprovincial level. We will start with the province of Córdoba for visualization purposes, and then extend the calculation to all provinces using an iterative approach.

In [ ]:
# Load Córdoba province departments (IDPROV = 14)
all_dptos = ee.FeatureCollection("projects/ee-pdt/assets/argentina/ar_prov_dpto")
cordoba_dptos = all_dptos.filter(ee.Filter.eq('IDPROV', "14"))

# Load built-up transition image and mask it
built_up_transition_masked = (
    ee.Image("projects/ee-pdt/assets/dynamicworld/built_up_transition_2021_2022")
    .clip(cordoba_dptos)
    .selfMask()
)

In [ ]:
# Compute built-up area per pixel in km²
pixel_area_km2 = ee.Image.pixelArea().divide(1e6)
built_up_area_km2 = built_up_transition_masked.multiply(pixel_area_km2).rename('built_up_km2')

# Sum built-up area per department
zonal_stats = built_up_area_km2.reduceRegions(
    collection=cordoba_dptos,
    reducer=ee.Reducer.sum().unweighted(),
    scale=10,
    crs='EPSG:4326'
)

In [ ]:
# Rename 'sum' to 'built_up_area_km2'
zonal_stats_renamed = zonal_stats.map(lambda f: f.set('built_up_area_km2', ee.Number(f.get('sum'))))

In [ ]:
# Function to calculate NBUT_2022
def compute_nbut(feature):
    built_area = ee.Number(feature.get('built_up_area_km2'))
    depto_area_km2 = feature.geometry().area().divide(1e6)
    nbut = ee.Algorithms.If(
        built_area.gt(0),
        built_area.divide(depto_area_km2).multiply(1000),
        0
    )
    return feature.set('NBUT_2022', nbut)

# apply the function
zonal_stats_nbut = zonal_stats_renamed.map(compute_nbut)

In [ ]:
# Convert to GeoDataFrame and extract desired columns
gdf_nbut_2022 = geemap.ee_to_gdf(zonal_stats_nbut).round(2)
df_nbut_2022 = gdf_nbut_2022[['IDDPTO', 'NBUT_2022']].round(2)

In [ ]:
# vissualize data.frame
init_notebook_mode(all_interactive=True)
df_nbut_2022

In [ ]:
# get basic statistics of data.frame
init_notebook_mode(all_interactive=True)
df_nbut_2022.describe()

#####  Mapping the NBUT 2020-2022 in Cordoba Province

We will create an interactive map to display df_nbut_2022 as a choropleth map and built_up_transition_masked image in Cordoba Province


In [ ]:
# Define color bins and values
# bins = [0, 0.5, 1.0, 1.5, 2.0, 2.5]
bins = [1.33, 1.93, 3.12, 4.53, 8.53,  14.08]
# colors = ["#fef0d9", "#fdcc8a", "#fc8d59", "#e34a33", "#b30000"]
colors =["#FDE725", "#FDE725", "#B8DE29",  "#5DC962",  "#21918C", "#440154"]

# Create a step colormap for legend
colormap = cm.StepColormap(colors=colors, vmin=bins[0], vmax=bins[-1], index=bins,
                           caption='Normalized Built-up Transitions (NBUT) 2020–2022 (/1000 km²)')

# Function to assign color to each feature based on NBUT_2022 value
def get_color(nbut):
    if nbut is None:
        return 'gray'
    for i in range(len(bins) - 1):
        if bins[i] <= nbut < bins[i + 1]:
            return colors[i]
    return colors[-1]

# Initialize map
m = folium.Map(location=[-31.3, -64.2], zoom_start=7, control_scale=True)

# Add the styled NBUT GeoJson layer
folium.GeoJson(
    gdf_nbut_2022,
    name='NBUT 2020-2022 (Custom Choropleth)',
    style_function=lambda feature: {
        'fillColor': get_color(feature['properties']['NBUT_2022']),
        'color': 'black',
        'weight': 0.3,
        'fillOpacity': 0.6,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['IDDPTO', 'NBUT_2022'],
        aliases=['Department ID:', 'NBUT (/1000 km²):'],
        localize=True
    )
).add_to(m)

# Add the colormap legend
colormap.add_to(m)

# Add built-up transition raster in black
vis_params_built_up = {
    'min': 0,
    'max': 1,
    'palette': ['#ffffff00', '#000000']
}
map_id_dict_built_up = built_up_transition_masked.getMapId(vis_params_built_up)

folium.raster_layers.TileLayer(
    tiles=map_id_dict_built_up['tile_fetcher'].url_format,
    attr='Earth Engine',
    name='Built-up Transition Raster (Black)',
    overlay=True,
    control=True,
    opacity=1
).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

In [ ]:
# display the map
m

#### Computing the NBUT for all provinces (2020-2022)

In [ ]:
# Load the shapefile to get province id list
shapefile_path = 'pdt/asthma_mortality/data/shp/ar_prov_dpto.shp'
gdf = gpd.read_file(shapefile_path)

# Get unique values of IDPROV
province_ids = gdf['IDPROV'].unique().tolist()

print(province_ids)

In [ ]:
# Load all province-department features
all_dptos = ee.FeatureCollection("projects/ee-pdt/assets/argentina/ar_prov_dpto")

In [ ]:
# Load the built-up transition image
built_up_transition = ee.Image("projects/ee-pdt/assets/dynamicworld/built_up_transition_2021_2022")

In [ ]:
# Compute area per pixel in km²
pixel_area_km2 = ee.Image.pixelArea().divide(1e6)

In [ ]:
# List to store DataFrames from each province
df_list = []

In [ ]:
# Loop through each province ID
for prov_id in province_ids:
    print("Processing province: ", prov_id)
    # ilter departments by province
    prov_dptos = all_dptos.filter(ee.Filter.eq('IDPROV', prov_id))

    # Clip and mask the built-up transition image
    built_up_masked = built_up_transition.clip(prov_dptos).selfMask()

    # Multiply by pixel area to get built-up area in km²
    built_up_area_km2 = built_up_masked.multiply(pixel_area_km2).rename('built_up_km2')

    # Zonal statistics: sum built-up area by department
    zonal_stats = built_up_area_km2.reduceRegions(
        collection=prov_dptos,
        reducer=ee.Reducer.sum().unweighted(),
        scale=10,
        crs='EPSG:4326'
    )

    # Rename 'sum' to 'built_up_area_km2'
    zonal_stats_renamed = zonal_stats.map(lambda f: f.set('built_up_area_km2', ee.Number(f.get('sum'))))

    # Compute NBUT_2022
    def compute_nbut(feature):
        built_area = ee.Number(feature.get('built_up_area_km2'))
        depto_area_km2 = feature.geometry().area().divide(1e6)
        nbut = ee.Algorithms.If(
            built_area.gt(0),
            built_area.divide(depto_area_km2).multiply(1000),
            0
        )
        return feature.set('NBUT_2022', nbut)

    zonal_stats_nbut = zonal_stats_renamed.map(compute_nbut)

    # Convert to GeoDataFrame and filter only required columns
    gdf = geemap.ee_to_gdf(zonal_stats_nbut).round(2)
    df = gdf[['IDDPTO', 'NBUT_2022']]

    # Append to list
    df_list.append(df)

In [ ]:
# Merge all provincial data into one DataFrame
df_nbut_2022_all = pd.concat(df_list, ignore_index=True)

In [ ]:
# visualize data.frame
init_notebook_mode(all_interactive=True)
df_nbut_2022_all

In [ ]:
# get basic info of the data.frame
df_nbut_2022_all.info()

In [ ]:
# test if df_nbut_2022_all["NBUT_2022"] for cordoba province is the same as df_nbut_2022["NBUT_2022"]

# Filter df_nbut_2022_all for Cordoba province (IDPROV = '14')
df_nbut_2022_all_cordoba = df_nbut_2022_all[df_nbut_2022_all['IDDPTO'].str.startswith('14')]

# Sort both dataframes by 'IDDPTO' to ensure comparison alignment
df_nbut_2022_all_cordoba_sorted = df_nbut_2022_all_cordoba.sort_values(by='IDDPTO').reset_index(drop=True)
df_nbut_2022_cordoba_sorted = df_nbut_2022.sort_values(by='IDDPTO').reset_index(drop=True)

# Test if the 'NBUT_2022' columns are the same
are_equal = (df_nbut_2022_all_cordoba_sorted['NBUT_2022'] == df_nbut_2022_cordoba_sorted['NBUT_2022']).all()

print(f"Are the 'NBUT_2022' values for Cordoba province the same?: {are_equal}")

##### Merge NBUT (2020-2022) with other features

In [ ]:
# load geopackage with PM2.5, Burned areas and other datasets
gdf = gpd.read_file("pdt/asthma_mortality/data/gpkg/tma_pm25_ba_pd_pdpm25_agrt_nwvt_nbut0020_2001_2022.gpkg")

In [ ]:
# Perform a left merge, preserving all rows from gdf
gdf_nbut_2022 = gdf.merge(df_nbut_2022_all, on='IDDPTO', how='left')

In [ ]:
# visualize gdf
init_notebook_mode(all_interactive=True)
gdf_nbut_2022.head()

In [ ]:
# check dataframe shape
gdf_nbut_2022.shape

In [ ]:
# Save dataset with NBUT (2005-2010) as other features as a gpkg file
gdf_nbut_2022.to_file("pdt/asthma_mortality/data/gpkg/tma_pm25_ba_pd_pdpm25_agrt_nwvt_nbut_2001_2022.gpkg", driver="GPKG")